In [1]:
import torch
from torch import nn
import numpy as np
from torchsummary import summary
import Lossv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(Lossv2.Loss().to(device), (6, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 768, 512]             224
            Conv2d-2         [-1, 32, 384, 512]             128
            Conv2d-3         [-1, 32, 384, 256]             128
               ELU-4         [-1, 32, 384, 256]               0
            Conv2d-5         [-1, 32, 384, 256]           1,056
            Conv2d-6         [-1, 32, 384, 256]             128
            Conv2d-7         [-1, 32, 384, 256]             128
               ELU-8         [-1, 32, 384, 256]               0
            Conv2d-9         [-1, 32, 384, 256]           1,056
           Conv2d-10         [-1, 32, 192, 256]             128
           Conv2d-11         [-1, 32, 192, 128]             128
              ELU-12         [-1, 32, 192, 128]               0
           Conv2d-13         [-1, 32, 384, 256]           1,056
        MaxPool2d-14         [-1, 32, 1

In [1]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import Lossv2
import generateLossImages

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./models/rest/", best_model_dir="./models/best/"):
    f_path = checkpoint_dir + 'checkpoint.pt'
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

def load_ckp(model, optimizer, checkpoint_fpath="./models/best/best_model.pt"):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['index'], checkpoint['min_lr'], checkpoint['max_lr'], checkpoint['steps'], checkpoint['step_size'], checkpoint['falling']

In [2]:

loss_fn = nn.L1Loss()
min_lr = 0.0003
max_lr = 0.001
decay = 0.8
steps = 300
falling = True
start_epoch = 0
start_index = 0
step_size = (max_lr-min_lr)/steps
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=0.96)

In [17]:
model = Lossv2.Loss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.96)
model, optimizer, start_epoch, start_index, min_lr, max_lr, steps, step_size, falling = load_ckp(model, optimizer)

In [3]:

running_loss = 0.
last_loss = 0.
lis = []
threshold = [0.24, 0.2, 0.16, 0.12, 0.09, 0.06, 0.04, 0.03, 0.02, 0.015, 0.01 -1] # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100
for epoch in range(start_epoch, epochs):

    training = generateLossImages.MakeIter(generateLossImages.get_image_pairs_transforms_with_loss(cupy=True, start=start_index if epoch == start_epoch else 0)) 
    val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training)

    for index, data in enumerate(training_loader):

        inputs, labels = data

        labels = labels.to("cuda")
        
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Gather data and report
        running_loss += loss.item()
        
        lis.append((f'{outputs.item():.3}',f'{labels.item():.3}'))
        if index % 71 == 70:
            # Zero your gradients for every batch!
            
            # Adjust learning weights
            last_loss = running_loss / 71 # loss per batch
            if last_loss < threshold[0] and flags[0]:
                threshold = threshold[1:]
                flags = flags[1:]
                flag1 = False
                min_lr *= threshold_decay
                max_lr *= threshold_decay
                step_size = (max_lr-min_lr)/steps
                falling = True
                optimizer.param_groups[-1]['lr'] = max_lr
                print(min_lr)
                print(max_lr)
                print(steps)
                print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
                }
                save_ckp(checkpoint, True)
            elif falling:
                optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
                if optimizer.param_groups[-1]['lr'] < min_lr:
                    falling = False
                    max_lr *= decay
                    min_lr *= decay
                    steps /= decay
                    step_size = (max_lr-min_lr)/steps
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)

            else: 
                optimizer.param_groups[-1]['lr'] += step_size
                if optimizer.param_groups[-1]['lr'] > max_lr:
                    falling = True


            
            optimizer.step()
            optimizer.zero_grad()
            print(optimizer.param_groups[-1]['lr'])
            print('  batch {} loss: {}'.format(index + 1, last_loss))
            print(lis)
            lis = []
            running_loss = 0

            if last_loss < record:
                record = last_loss
                if last_loss < 0.08:
                    checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
                }
                    save_ckp(checkpoint, True)

                

C:/Users/Rani/Desktop/ai_training_immages/5/_DSC0001.NEF


c:\Users\Rani\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


0.0009976666666666667
  batch 71 loss: 0.29674743892441335
[('0.525', '0.0'), ('0.521', '0.0'), ('0.519', '0.0'), ('0.528', '0.0'), ('0.513', '0.0'), ('0.501', '0.0'), ('0.525', '0.0'), ('0.525', '0.0'), ('0.515', '0.0'), ('0.521', '0.0'), ('0.508', '0.999'), ('0.509', '0.999'), ('0.513', '0.999'), ('0.501', '0.999'), ('0.501', '0.999'), ('0.515', '0.999'), ('0.513', '0.999'), ('0.502', '0.999'), ('0.507', '0.999'), ('0.511', '0.999'), ('0.502', '0.999'), ('0.505', '0.999'), ('0.522', '0.2'), ('0.522', '0.2'), ('0.522', '0.2'), ('0.501', '0.2'), ('0.5', '0.2'), ('0.524', '0.2'), ('0.524', '0.2'), ('0.52', '0.2'), ('0.524', '0.2'), ('0.519', '0.2'), ('0.518', '0.2'), ('0.519', '0.6'), ('0.521', '0.6'), ('0.508', '0.6'), ('0.5', '0.6'), ('0.5', '0.6'), ('0.519', '0.6'), ('0.516', '0.6'), ('0.507', '0.6'), ('0.52', '0.6'), ('0.522', '0.6'), ('0.507', '0.6'), ('0.507', '0.6'), ('0.519', '0.4'), ('0.522', '0.4'), ('0.513', '0.4'), ('0.5', '0.4'), ('0.5', '0.4'), ('0.521', '0.4'), ('0.519', 